In [ ]:
import torch
import numpy as np
from matplotlib import pyplot as plt

%run '/content/drive/MyDrive/Colab Notebooks/CPSC 490/model/log_utils.ipynb'
%run '/content/drive/MyDrive/Colab Notebooks/CPSC 490/model/networks.ipynb'

Mounted at /content/drive
Mounted at /content/drive


In [ ]:
class ClassificationModel(object):
    '''
    Classification model class that supports VGG11 and ResNet18 encoders

    Arg(s):
        encoder_type : str
            encoder options to build: vggnet11, resnet18, etc.
        device : torch.device
            device to run model on
    '''

    def __init__(self,
                 encoder_type,
                 device=torch.device('cuda')):

        self.device = device

        # TODO: Instantiate VGG11 and ResNet18 encoders and decoders based on
        # https://arxiv.org/pdf/1409.1556.pdf
        # https://arxiv.org/pdf/1512.03385.pdf
        # Decoder should use
        # https://pytorch.org/docs/stable/generated/torch.nn.Sequential.html
        input_channels = 3
        if encoder_type == 'vggnet11':
            self.encoder = VGGNet11Encoder(input_channels=input_channels,
                                           n_filters=[64, 128, 256, 512, 512])
            self.decoder = torch.nn.Sequential(torch.nn.Flatten(),
                                               torch.nn.Linear(512, 4096),
                                               torch.nn.ReLU(True),
                                               torch.nn.Dropout(),
                                               torch.nn.Linear(4096, 4096),
                                               torch.nn.ReLU(True),
                                               torch.nn.Dropout(),
                                               torch.nn.Linear(4096, 1000))
        elif encoder_type == 'resnet18':
            self.encoder = ResNet18Encoder(input_channels=input_channels,
                                           n_filters=[64, 64, 128, 256, 512])
            self.decoder = torch.nn.Sequential(torch.nn.AdaptiveAvgPool2d((1,1)),
                                              torch.nn.Flatten(1, -1),
                                              torch.nn.Linear(512, 10))
        else:
            raise ValueError('Unsupported encoder type: {}'.format(encoder_type))
        self.encoder_type = encoder_type

    def transform_input(self, images):
        '''
        Transforms input based on model arguments and settings

        Arg(s):
            images : torch.Tensor[float32]
                N x C x H x W images
        Returns:
            torch.Tensor[float32] : transformed N x C x H x W images
        '''

        # TODO: Perform normalization based on
        # https://arxiv.org/pdf/1409.1556.pdf
        # https://arxiv.org/pdf/1512.03385.pdf

        if self.encoder_type == 'vggnet11':
            images = images / 255.0
        elif self.encoder_type == 'resnet18':
            normalize = torch.nn.transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
            images = normalize(images)

        return images

    def forward(self, image):
        '''
        Forwards inputs through the network

        Arg(s):
            image : torch.Tensor[float32]
                N x 3 x H x W image
        Returns:
            torch.Tensor[float32] : N x K predicted class confidences
        '''

        # TODO: Implement forward function
        latent_space, intermediate = self.encoder(image)
        output = self.decoder(latent_space)

        return output

    def compute_loss(self, output, label):
        '''
        Compute cross entropy loss

        Arg(s):
            output : torch.Tensor[float32]
                N x K predicted class confidences
            label : torch.Tensor[int]
                ground truth class labels
        Returns:
            float : loss averaged over the batch
            dict[str, float] : dictionary of loss related tensors
        '''

        # TODO: Compute cross entropy loss
        loss = torch.nn.functional.cross_entropy(
            input=output,
            target=label)

        loss_info = {
            'loss' : loss.item()
        }

        return loss, loss_info

    def parameters(self):
        '''
        Returns the list of parameters in the model

        Returns:
            list[torch.Tensor[float32]] : list of parameters
        '''

        return list(self.encoder.parameters()) + list(self.decoder.parameters())

    def train(self):
        '''
        Sets model to training mode
        '''
        self.encoder.train()
        self.decoder.train()

    def eval(self):
        '''
        Sets model to evaluation mode
        '''

        self.encoder.eval()
        self.decoder.eval()

    def to(self, device):
        '''
        Move model to a device

        Arg(s):
            device : torch.device
                device to use
        '''

        self.device = device

        self.encoder.to(self.device)
        self.decoder.to(self.device)

        # TODO: Move encoder and decoder to device

    def data_parallel(self):
        '''
        Allows multi-gpu split along batch
        '''

        # TODO: Wrap encoder and decoder in
        # https://pytorch.org/docs/stable/generated/torch.nn.DataParallel.html

        self.encoder = torch.nn.DataParallel(self.encoder)
        self.decoder = torch.nn.DataParallel(self.decoder)

    def restore_model(self, restore_path, optimizer=None):
        '''
        Loads weights from checkpoint

        Arg(s):
            restore_path : str
                lists of paths to model weights
            optimizer : torch.optim or None
                current optimizer
        Returns:
            int : training step
            torch.optim : restored optimizer or None if no optimizer is passed in
        '''

        # TODO: Restore the weights from checkpoint
        # Encoder and decoder are keyed using 'encoder_state_dict' and 'decoder_state_dict'
        # If optimizer is given, then save its parameters using key 'optimizer_state_dict'
        checkpoint = torch.load(restore_path)
        self.encoder.load_state_dict(checkpoint['encoder_state_dict'])
        self.decoder.load_state_dict(checkpoint['decoder_state_dict'])

        if optimizer:
            optimizer.load_state_dict(checkpoint['optimizer_state_dict'])

        return checkpoint['step'], optimizer

    def save_model(self, checkpoint_path, step, optimizer=None):
        '''
        Save weights of the model to checkpoint path

        Arg(s):
            checkpoint_path : str
                list path to save checkpoint
            step : int
                current training step
            optimizer : torch.optim
                optimizer
        '''

        # TODO: Save the weights into checkpoint
        # Encoder and decoder are keyed using 'encoder_state_dict' and 'decoder_state_dict'
        # If optimizer is given, then save its parameters using key 'optimizer_state_dict'
        torch.save({
            'step': step,
            'encoder_state_dict': self.encoder.state_dict(),
            'decoder_state_dict': self.decoder.state_dict(),
            'optimzier_state_dict': optimizer.state_dict() if optimizer else None
        },
        checkpoint_path)

    def log_summary(self,
                    summary_writer,
                    tag,
                    step,
                    image,
                    output,
                    ground_truth,
                    scalars={},
                    n_image_per_summary=16):
        '''
        Logs summary to Tensorboard

        Arg(s):
            summary_writer : SummaryWriter
                Tensorboard summary writer
            tag : str
                tag that prefixes names to log
            step : int
                current step in training
            image : torch.Tensor[float32] 640 x 480
                image at time step
            output : torch.Tensor[float32]
                N
            label : torch.Tensor[float32]
                ground truth force measurements or ground truth bounding box and force measurements
            scalars : dict[str, float]
                dictionary of scalars to log
            n_image_per_summary : int
                number of images to display
        '''

        grid_size = int(np.sqrt(n_image_per_summary))
        n_image_per_summary = int(grid_size * grid_size)

        with torch.no_grad():

            image_summary = image[0:n_image_per_summary, ...] / 255.0

            # TODO: Move image_summary to CPU using cpu()
            image_summary = image_summary.cpu()

            # TODO: Convert image_summary to numpy using numpy() and swap dimensions from NCHW to NHWC
            image_summary = image_summary.numpy().transpose((0,2,3,1))

            # TODO: Create plot figure of size n x n using log_utils
            grid_size = int(n_image_per_summary ** 0.5)
            images_display = []
            subplot_titles = []
            for row in range(grid_size):
                idx_start = row * grid_size
                idx_end = (row+1) * grid_size
                images_display.append(image_summary[idx_start:idx_end])
                subplot_titles.append([f'output={output[i]}\nlabel={ground_truth[i]}' for i in range(idx_start, idx_end)])
            figure = plot_images(images_display, grid_size, grid_size, subplot_titles)

            # TODO: Log image summary to Tensorboard with <tag>_image as its summary tag name using
            # https://pytorch.org/docs/stable/tensorboard.html#torch.utils.tensorboard.writer.SummaryWriter.add_figure
            summary_writer.add_figure(f'{tag}_image', figure, global_step=step)

            plt.tight_layout()

            plt.cla()
            plt.clf()
            plt.close()

            # TODO: Log scalars to Tensorboard with <tag>_<name> as its summary tag name using
            # https://pytorch.org/docs/stable/tensorboard.html#torch.utils.tensorboard.writer.SummaryWriter.add_scalar
            for (name, value) in scalars.items():
                summary_writer.add_scalar(f'{tag}_{name}', value, step)

NameError: name 'torch' is not defined